# Agent Behaviours (ABM)

In this Notebook all the behaviours that the spatial agents can perform are defined.The number of agents taken is considered to be one arbitarily one for the purpose of studying the rowth pattern definitively  
<br/> The following Agent Behaviours are planned:

## 1. Agent initial Position(Birth):
* Search(2D-3D)Space according to Stencil and the Space program and assigning the initial position
* Attraction / Repulsion movement of initial poostions of the agents based on the closeness relationships 

## 2. Occupy:
* (2D) Squareness growth: neighbours of neighbours get higher preference
* (2D) Rectillinear Polyomino growth: growth pattern is in a combination of squares [https://en.wikipedia.org/wiki/Polyomino#:~:text=A%20polyomino%20is%20a%20plane,of%20the%20regular%20square%20tiling.]
* (2D) Organic Polyomino growth : Growth is pure merit based only (holes are avoided)
* (3D) Cuboidal growth: neighbours of neighbours get higher preference in a cuboid
* (3D) Rectillinear Polyomino growth: growth pattern is in a combination of cubes
* (3D) Organic Polyomino growth : Growth is pure merit based only (holes are avoided)

## 3. Growth Lattice modification:
* (2D),(3D) Attraction: The base lattice gets modified with additional values based on graph distances between the selected agents
   making them grow towards each other
* (2D),(3D) Repulsion: The base lattice gets modified with additional values based on graph distances between the selected agents making them grow away from each other

## 4. Negotiation:??

## 5. Unoccupy:??

##  Initialization
Loading the necessary Libraries for the notebook

In [1]:
import os
import sys
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import Widget_for_display as wid
import Stencils as stn
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx

In [2]:
stencil = stn.stencil_von_neumann
print(stencil)

<function stencil_von_neumann at 0x0000023310BC65E0>


## Loading the Base Lattice ( Availability Lattice)
Since the stencil does not work for corner voxels the sides and the top and bottom voxels are made unavailable 

In [3]:
unit = 3
#Dummy Lattice for testing
Dim = 15
index = Dim -1
bounds = np.ones((Dim,Dim), dtype=int)
bounds[0,:] =0
bounds[:,0] =0
bounds[:,index] =0
bounds[index,:] =0
print (bounds)
init = np.tile(bounds,(Dim,1,1))
print(init.shape)
avail_lattice = tg.to_lattice(np.copy(init), init.shape)
init_avail_lattice = tg.to_lattice(np.copy(init), init.shape)
#print (init_avail_lattice)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
(15, 15, 15)


## Loading the Enviornment Lattices

In [4]:
#Dummy Lattice for testing

In [5]:
enviornment_lattice = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')/10
#print(enviornment_lattice)

In [6]:
one_neighbour_factor = 1
two_neighbour_factor = 2
three_neighbour_factor = 3
four_neighbour_factor = 4

## Define the Neighborhoods for the behaviours (Stencils)

In [7]:
# creating neighborhood definition
stencil = tg.create_stencil("von_neumann", 1, 1)
#print(stencil)

In [8]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

In [9]:
# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 
#print(stencil_squareness_von)

In [10]:
stencil_full_floor = tg.create_stencil("moore",int(Dim/2) )*0
stencil_full_floor[:,:,int(Dim/2)] = 1
stencil_full_floor.set_index([0,0,0], 0)
#print(stencil_full_floor)

In [11]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

In [12]:
stencil_full_lattice = tg.create_stencil("moore",int(Dim/2) )
stencil_full_lattice.set_index([0,0,0], 0)
#print(stencil_full_lattice)

## Vizualising the stencils 

In [13]:
Stencil_to_viz = stencil_squareness_t
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = np.array(Stencil_to_viz.shape) + 1
# The bottom left corner of the data set
grid.origin = [0,0,0]
# These are the cell sizes along each axis
grid.spacing = [1,1,1]

# Add the data values to the cell data
grid.cell_arrays["values"] = Stencil_to_viz.flatten(order="F")  # Flatten the stencil
threshed = grid.threshold([0.9, 1.1])

# adding the voxels: light red
p.add_mesh(threshed, show_edges=True, color="#ff8fa3", opacity=0.3)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(6.361720043489708, 6.361720043489708, 6.361720043489708),
 (1.5, 1.5, 1.5),
 (0.0, 0.0, 1.0)]

## Distance Field Construction

In [14]:
# find the number of all voxels
vox_count = avail_lattice.size 

# initialize the adjacency matrix
adj_mtrx = np.zeros((vox_count,vox_count))

# extract the neighbourhood of all voxels
# all_vox_neighs = avail_lattice.find_neighbours(stencil)

# Finding the index of the available voxels in avail_lattice
avail_index = np.array(np.where(avail_lattice == 1)).T

# fill the adjacency matrix using the list of all neighbours
for vox_loc in avail_index:
    # find the 1D id
    vox_id = np.ravel_multi_index(vox_loc, avail_lattice.shape)
    # retrieve the list of neighbours of the voxel based on the stencil
    vox_neighs = avail_lattice.find_neighbours_masked(stencil, loc = vox_loc)
    # iterating over the neighbours
    for neigh in vox_neighs:
        # setting the entry to one
        adj_mtrx[vox_id, neigh] = 1.0

# construct the graph 
g = nx.from_numpy_array(adj_mtrx)


In [15]:
# compute the distance of all voxels to all voxels using floyd warshal algorithm
#dist_mtrx = nx.floyd_warshall_numpy(g)

In [16]:

#csv_flyod_warshall = np.savetxt('data.csv', dist_mtrx, delimiter=',')

flyod_warshall_matrix =np.genfromtxt('data.csv',delimiter=',')
dist_mtrx = flyod_warshall_matrix

## Constructing Distance matrix to the agent origin

In [17]:
# select the corresponding row in the matrix
myagent_attractor_one_dist = flyod_warshall_matrix[682]

# find the maximum valid value
max_valid = np.ma.masked_invalid(myagent_attractor_one_dist).max()
print(max_valid)
# set the infinities to one more than the maximum valid values
myagent_attractor_one_dist[myagent_attractor_one_dist == np.inf] = max_valid + 1

# mapping the values from (0, max) to (1, 0)
dist_flat_one = 1 - myagent_attractor_one_dist / np.max(myagent_attractor_one_dist)
print(dist_flat_one)
# constructing the lattice
ent_acc_lattice_1 = tg.to_lattice(dist_flat_one.reshape(avail_lattice.shape), avail_lattice)

32.0
[0.      0.      0.      0.      0.      0.      0.      0.      0.
 0.      0.      0.      0.      0.      0.      0.      0.71875 0.75
 0.78125 0.8125  0.84375 0.875   0.875   0.875   0.84375 0.8125  0.78125
 0.75    0.71875 0.      0.      0.6875  0.71875 0.75    0.78125 0.8125
 0.84375 0.875   0.84375 0.8125  0.78125 0.75    0.71875 0.6875  0.
 0.      0.65625 0.6875  0.71875 0.75    0.78125 0.8125  0.84375 0.8125
 0.78125 0.75    0.71875 0.6875  0.65625 0.      0.      0.625   0.65625
 0.6875  0.71875 0.75    0.78125 0.8125  0.78125 0.75    0.71875 0.6875
 0.65625 0.625   0.      0.      0.59375 0.625   0.65625 0.6875  0.71875
 0.75    0.78125 0.75    0.71875 0.6875  0.65625 0.625   0.59375 0.
 0.      0.5625  0.59375 0.625   0.65625 0.6875  0.71875 0.75    0.71875
 0.6875  0.65625 0.625   0.59375 0.5625  0.      0.      0.53125 0.5625
 0.59375 0.625   0.65625 0.6875  0.71875 0.6875  0.65625 0.625   0.59375
 0.5625  0.53125 0.      0.      0.5     0.53125 0.5625  0.59375 0.6

In [18]:
# select the corresponding row in the matrix
myagent_attractor_two_dist = dist_mtrx[1200]

# find the maximum valid value
max_valid = np.ma.masked_invalid(myagent_attractor_two_dist).max()
print(max_valid)
# set the infinities to one more than the maximum valid values
myagent_attractor_two_dist[myagent_attractor_two_dist == np.inf] = max_valid + 1

# mapping the values from (0, max) to (1, 0)
dist_flat_two = 1 - myagent_attractor_two_dist / np.max(myagent_attractor_two_dist)
print(dist_flat_two)
# constructing the lattice
ent_acc_lattice_2 = tg.to_lattice(dist_flat_two.reshape(avail_lattice.shape), avail_lattice)

32.0
[0.      0.      0.      0.      0.      0.      0.      0.      0.
 0.      0.      0.      0.      0.      0.      0.      0.71875 0.6875
 0.65625 0.625   0.59375 0.5625  0.53125 0.5     0.46875 0.4375  0.40625
 0.375   0.34375 0.      0.      0.75    0.71875 0.6875  0.65625 0.625
 0.59375 0.5625  0.53125 0.5     0.46875 0.4375  0.40625 0.375   0.
 0.      0.78125 0.75    0.71875 0.6875  0.65625 0.625   0.59375 0.5625
 0.53125 0.5     0.46875 0.4375  0.40625 0.      0.      0.8125  0.78125
 0.75    0.71875 0.6875  0.65625 0.625   0.59375 0.5625  0.53125 0.5
 0.46875 0.4375  0.      0.      0.8125  0.8125  0.78125 0.75    0.71875
 0.6875  0.65625 0.625   0.59375 0.5625  0.53125 0.5     0.46875 0.
 0.      0.8125  0.78125 0.75    0.71875 0.6875  0.65625 0.625   0.59375
 0.5625  0.53125 0.5     0.46875 0.4375  0.      0.      0.78125 0.75
 0.71875 0.6875  0.65625 0.625   0.59375 0.5625  0.53125 0.5     0.46875
 0.4375  0.40625 0.      0.      0.75    0.71875 0.6875  0.65625 0.625
 

##  Agents

In the agent based system, when the stencil reaches its limit the full lattice/full floor depending on 2D or 3D  is considered in the simulation as a stencil.

### Define the Agents Class

In [19]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
        
        
        
        
        
        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
    
    
    
    
      # definition of 3d occupying method for agents
    def one_neighbour_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 25 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[3] + neighs_availibility[9]+1
        two = neighs_availibility[0] + neighs_availibility[2] + neighs_availibility[4] + neighs_availibility[10] 
        three = neighs_availibility[1] + neighs_availibility[5]  + neighs_availibility[11] +1
        four = neighs_availibility[0] + neighs_availibility[4] + neighs_availibility[6] + neighs_availibility[12] 
        five = neighs_availibility[1] + neighs_availibility[3] + neighs_availibility[5] + neighs_availibility[7] 
        six = neighs_availibility[2] + neighs_availibility[4] + neighs_availibility[8] + neighs_availibility[13] 
        seven = neighs_availibility[3] + neighs_availibility[7] + neighs_availibility[14] +1
        eight = neighs_availibility[4] + neighs_availibility[6] + neighs_availibility[8] + neighs_availibility[15] 
        nine = neighs_availibility[5] + neighs_availibility[7]  + neighs_availibility[16] +1
        ten = neighs_availibility[0] + neighs_availibility[10]  + neighs_availibility[12] + neighs_availibility[17]
        eleven = neighs_availibility[1] + neighs_availibility[9] + neighs_availibility[11] + neighs_availibility[18]
        twelve = neighs_availibility[2] + neighs_availibility[10] + neighs_availibility[13] + neighs_availibility[19]
        thirteen = neighs_availibility[3] + neighs_availibility[9] + neighs_availibility[14] + neighs_availibility[20]
        fourteen = neighs_availibility[5] + neighs_availibility[11] + neighs_availibility[16] + neighs_availibility[22]
        fifteen = neighs_availibility[6] + neighs_availibility[12] + neighs_availibility[15] + neighs_availibility[23]
        sixteen = neighs_availibility[7] + neighs_availibility[14] + neighs_availibility[16] + neighs_availibility[24]
        seventeen = neighs_availibility[8] + neighs_availibility[13] + neighs_availibility[15] + neighs_availibility[25]
        eighteen = neighs_availibility[9] + neighs_availibility[18] + neighs_availibility[20] +1
        nineteen = neighs_availibility[10] + neighs_availibility[17] + neighs_availibility[19] + neighs_availibility[21]
        twenty = neighs_availibility[11] + neighs_availibility[18] + neighs_availibility[22] +1
        twentyone = neighs_availibility[12] + neighs_availibility[17] + neighs_availibility[21] + neighs_availibility[23]
        twentytwo = neighs_availibility[18] + neighs_availibility[20] + neighs_availibility[22] + neighs_availibility[24]
        twentythree = neighs_availibility[13] + neighs_availibility[19] + neighs_availibility[21] + neighs_availibility[25]
        twentyfour = neighs_availibility[14] + neighs_availibility[20] + neighs_availibility[24] +1
        twentyfive = neighs_availibility[15] + neighs_availibility[21] + neighs_availibility[23] + neighs_availibility[20]
        twentysix =  neighs_availibility[16] + neighs_availibility[22] + neighs_availibility[24] +1
        
        neighbourhood_details = [one,two,three,four,five,six,seven,eight,nine,ten,eleven,twelve,thirteen,fourteen,fifteen,sixteen,
                                    seventeen,eighteen,nineteen,twenty,twentyone,twentytwo,twentythree,twentyfour,twentyfive]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 5:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 6:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            elif neighbourhood_condition == 7:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + three_neighbour_factor
            elif neighbourhood_condition == 8:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + four_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

## Initializing Agents 

In [20]:
# occ_lattice intilization
occ_lattice = avail_lattice*0 -1
#print(occ_lattice)
# Finding the index of the available voxels in avail_lattice
avail_flat = avail_lattice.flatten()
avail_index = np.array(np.where(avail_lattice == 1)).T

# Randomly choosing one available voxels
agn_num = 1
select_id = np.random.choice(len(avail_index), agn_num)
agn_origins = tuple(avail_index[select_id].flatten())
#stencil_squareness_t

select_id_2 = np.random.choice(len(avail_index), agn_num)
agn_origins_2 = tuple(avail_index[select_id_2].flatten())

select_id_3 = np.random.choice(len(avail_index), agn_num)
agn_origins_3 = tuple(avail_index[select_id_2].flatten())
print(select_id)
print(select_id_2)
#print(select_id_2)
myagent = agent(agn_origins, stencil_squareness_t, select_id)
myagent_2 = agent(agn_origins_2, stencil_cuboid, select_id_2)
myagent_3 = agent(agn_origins_3, stencil_squareness_t, select_id_3)
#stencil_squareness_t,stencil_cuboid
#print(select_id_2)

[765]
[73]


In [21]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent
    

In [22]:
#Agent_2D = Agent_initilization 
Agent_one=initialize_agents_random_origin (stencil_squareness_von_t,avail_lattice )
Agent_two=initialize_agents_random_origin (stencil,avail_lattice )
print(Agent_one)

In [23]:
select_id_attractor_one = 682
agn_origins_attractor_one = tuple(avail_index[select_id_attractor_one].flatten())
myagent_attractor_one = agent(agn_origins_attractor_one, stencil_cuboid, select_id_attractor_one)
print(agn_origins_attractor_one)

(4, 1, 7)


In [24]:
select_id_attractor_two = 1200
agn_origins_attractor_two = tuple(avail_index[select_id_attractor_two].flatten())
myagent_attractor_two = agent(agn_origins_attractor_two, stencil_cuboid, select_id_attractor_two)

In [25]:
print(agn_origins_attractor_two)

(7, 2, 5)


## Define Environment Class

In [26]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility * 0
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
    
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()

## Creating the Enviornment

In [27]:
# name the lattices myagent_attractor_one
env_lattices = {"availibility": avail_lattice,"enviornment": enviornment_lattice}
env_lattices_attractors_one = {"availibility": avail_lattice,"enviornment": ent_acc_lattice_1}
env_lattices_attractors_two = {"availibility": avail_lattice,"enviornment": ent_acc_lattice_2}
# initiate the environment
env = environment(env_lattices, myagent,50,"one_neighbour_occupy_squareness_moore")
env_2 = environment(env_lattices, myagent_2,100,"one_neighbour_occupy_cubish_agents")
env_3 = environment(env_lattices, myagent_3,50,"one_neighbour_occupy_squareness_moore")
env_4 = environment(env_lattices, Agent_one,50,"one_neighbour_occupy_squareness_von_neumann")
env_5 = environment(env_lattices, Agent_two,100,"random_occupy_cubish_von_neumann_agents")
env_6 = environment(env_lattices_attractors_one, myagent_attractor_one,100,"one_neighbour_occupy_cubish_agents")
env_7 = environment(env_lattices_attractors_two, myagent_attractor_two,100,"one_neighbour_occupy_cubish_agents")

In [28]:
print(env_2.agents)

## Run the Simulation

In [29]:
env_availability_viz = []
env_list =[env_6,env_7]
number_steps = max(map(lambda e:e.number_of_iterations,env_list))
for a in range(number_steps):
    # print(env.availibility)
    # print(env.agent_origin)

    for e in env_list:
        if a < e.number_of_iterations :
            #print(a)
            #print(e.number_of_iterations)
            if e.method_name == "one_neighbour_occupy_squareness_moore":
                e.one_neighbour_occupy_squareness_moore()
                
            elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                e.one_neighbour_occupy_cubish_agents()
                
            elif e.method_name == "random_occupy_squareness_agents" :
                e.random_occupy_squareness_agents()
            
            elif e.method_name == "random_occupy_cubish_agents" :
                e.random_occupy_cubish_agents()
                
                
            elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                e.random_occupy_cubish_von_neumann_agents()                
                
                
            elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                e.one_neighbour_occupy_squareness_von_neumann()                
                
                
            env_availability_viz.append(e.availibility-1)
    

## Vizualize the Simulation

In [30]:
p = pv.Plotter(notebook=True)

base_lattice = env_availability_viz[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = env_availability_viz[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.0, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = sum(map(lambda e:e.number_of_iterations,env_list))
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)


[1 1 1]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(50.97777478867205, 50.97777478867205, 50.97777478867205),
 (22.0, 22.0, 22.0),
 (0.0, 0.0, 1.0)]

## Empty Cells Resolution ??

## Questions?

1. Can stencils which are unequal be defined using arrays ? 
2. Is there a better way of checking the neighbourhood of each cell in stencil than the manual process
3. I have removed the for loop in the defninitions maybe there is a way of not defining a for loop in the agent class to enable a single agent 
4. Is it correct to consider full floor/full lattice as a neighbourhood when the stencil cannot find the necessary neighbourhoods or am I going wrong somewhere ?
5. Agent origin location should be a seperate class in itself whose output can be fed into the agent initilization 
6. Placement of Shafts and corridors 

## Saving the Lattices in csv